In [4]:
import os, re, sys
from pdf2image import convert_from_path
from fpdf import FPDF

from dotenv import find_dotenv, load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
import boto3
client = boto3.client('textract', region_name='us-east-1', aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"), aws_secret_access_key=os.getenv("AWS_SECRET_KEY"))

In [7]:
# Store Pdf with convert_from_path function
images = convert_from_path('OCR-challenge/tough.pdf', poppler_path=r"C:\1-All-files\RapidLabs\pdftojson-new\poopler\Release-24.07.0-0\poppler-24.07.0\Library\bin")
 
for i in range(len(images)):
      # Save pages as images in the pdf
    images[i].save('OCR-challenge/pdf2img/page'+ str(i) +'.jpg', 'JPEG')

In [9]:
def sort_key(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

file_list = os.listdir("OCR-challenge/new_cropped_images")
sorted_file_list = sorted(file_list, key=sort_key)

for i, img in enumerate(sorted_file_list):
    with open(f'OCR-challenge/new_cropped_images/{img}', 'rb') as image:
        bytes_img = bytearray(image.read())
    print("--------------------", img, i)
    response = client.detect_document_text(
        Document={'Bytes': bytes_img}
    )

    text = ""
    for item in response["Blocks"]:
        if item["BlockType"] == "LINE":
            text = text + " "+item["Text"]

    with open(f"OCR-challenge/img2text/page{i}.txt", "w") as text_file:
        text_file.write(text)

pdf = FPDF()

text_list = os.listdir("OCR-challenge/img2text")
sorted_text_list = sorted(text_list, key=sort_key)

for i, text in enumerate(sorted_text_list):
    print("--------------------", text, i)

    # Add a page
    pdf.add_page()
    
    # set style and size of font
    # that you want in the pdf
    pdf.set_font("Arial", size = 8)
    
    # open the text file in read mode
    f = open(f"OCR-challenge/img2text/{text}", "r", encoding="latin-1")
    
    # insert the texts in pdf
    for x in f:
        pdf.cell(180, 5, txt = x, ln = 1, align = 'C')

    f.close()
# save the pdf with name .pdf
pdf.output('OCR-challenge/text2pdf/new.pdf', "F")

-------------------- section_0_0.jpg 0
-------------------- section_0_1.jpg 1
-------------------- section_0_2.jpg 2
-------------------- section_1_0.jpg 3
-------------------- section_1_1.jpg 4
-------------------- section_1_2.jpg 5
-------------------- section_2_0.jpg 6
-------------------- section_2_1.jpg 7
-------------------- section_2_2.jpg 8
-------------------- page0.txt 0
-------------------- page1.txt 1
-------------------- page2.txt 2
-------------------- page3.txt 3
-------------------- page4.txt 4
-------------------- page5.txt 5
-------------------- page6.txt 6
-------------------- page7.txt 7
-------------------- page8.txt 8
-------------------- page9.txt 9
-------------------- page10.txt 10
-------------------- page11.txt 11
-------------------- page12.txt 12
-------------------- page13.txt 13
-------------------- page14.txt 14
-------------------- page15.txt 15
-------------------- page16.txt 16


''

In [10]:
import textwrap

max_width = 10

for i, text in enumerate(sorted_text_list):
    print("--------------------", text, i)

    # Add a page
    pdf.add_page()
    
    # set style and size of font
    # that you want in the pdf
    pdf.set_font("Arial", size = 8)
    
    # # open the text file in read mode
    # f = open(f"OCR-challenge/img2text/{text}", "r", encoding="latin-1")
    
    # # insert the texts in pdf
    # for x in f:
    #     pdf.cell(180, 5, txt = x, ln = 1, align = 'C')

    # f.close()

    # Open the text file in read mode
    with open(f"OCR-challenge/img2text/{text}", "r", encoding="latin-1") as f:
        # Read the entire file content
        file_content = f.read()
        
        # Wrap the text at the specified width
        wrapped_text = textwrap.fill(file_content, width=max_width)
        
        # Insert the wrapped text into the PDF using MultiCell
        pdf.multi_cell(0, 10, txt=wrapped_text, align='L')
# save the pdf with name .pdf
pdf.output('OCR-challenge/text2pdf-new/new.pdf', "F")

-------------------- page0.txt 0
-------------------- page1.txt 1
-------------------- page2.txt 2
-------------------- page3.txt 3
-------------------- page4.txt 4
-------------------- page5.txt 5
-------------------- page6.txt 6
-------------------- page7.txt 7
-------------------- page8.txt 8
-------------------- page9.txt 9
-------------------- page10.txt 10
-------------------- page11.txt 11
-------------------- page12.txt 12
-------------------- page13.txt 13
-------------------- page14.txt 14
-------------------- page15.txt 15
-------------------- page16.txt 16


''

In [12]:
from PIL import Image

# Open the image file
image_path = f"OCR-challenge/new_cropped_images/{sorted_file_list[0]}"
img = Image.open(image_path)

# Define the number of sections (rows and columns) you want to divide the image into
num_rows = 3
num_cols = 3

# Get the dimensions of the image
img_width, img_height = img.size

# Calculate the size of each section
section_width = img_width // num_cols
section_height = img_height // num_rows

# Iterate over the rows and columns to crop and save each section
for row in range(num_rows):
    for col in range(num_cols):
        # Define the box to crop (left, upper, right, lower)
        left = col * section_width
        upper = row * section_height
        right = (col + 1) * section_width
        lower = (row + 1) * section_height
        box = (left, upper, right, lower)

        # Crop the image
        section = img.crop(box)

        # Save the cropped section as a new image
        section.save(f'OCR-challenge/new_cropped_images/section_{row}_{col}.jpg')

print("Image divided and sections saved successfully.")


Image divided and sections saved successfully.
